<a href="https://colab.research.google.com/github/BOD-27/GP/blob/main/LastEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mtcnn
!pip install keras-facenet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10369 sha256=ec6664471229a0c36a9eab28eee36bc7875460f0b0af4aa0871ff0e3e33a40db
  Stored in directory: /root/.cache/pip/wheels/1d/d8/a9/85cf04ea29321d2afcb82c0caaafdca9195385f9d68cbc7185
Successfully built keras-facenet


In [3]:
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
from scipy.spatial.distance import cosine
import re
from collections import Counter

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [6]:
# Create a directory in Google Drive to save embeddings
embedding_save_directory = '/content/drive/My Drive/Embedding'
if not os.path.exists(embedding_save_directory):
    os.makedirs(embedding_save_directory)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!git clone https://github.com/BOD-27/GP

Cloning into 'GP'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 34 (delta 11), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (34/34), 59.07 MiB | 26.08 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [7]:
!unzip /content/GP/FinalTest_dataset.zip

Archive:  /content/GP/FinalTest_dataset.zip
   creating: FinalTest_dataset/Abdo/
  inflating: FinalTest_dataset/Abdo/Abdo.jpg  
  inflating: FinalTest_dataset/Abdo/Abdo1.png  
  inflating: FinalTest_dataset/Abdo/Abdo2.png  
  inflating: FinalTest_dataset/Abdo/Abdo3.png  
  inflating: FinalTest_dataset/Abdo/Abdo4.png  
  inflating: FinalTest_dataset/Abdo/Abdo5.png  
  inflating: FinalTest_dataset/Abdo/Abdo6.png  
   creating: FinalTest_dataset/Ahmed/
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (1).jpg  
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (1).png  
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (2).png  
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (3).png  
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (4).png  
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (5).png  
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (6).png  
   creating: FinalTest_dataset/AhmedMoawd/
  inflating: FinalTest_dataset/AhmedMoawd/AhmedMoawd.jpg  
  inflating: FinalTest_dataset/AhmedMoawd/AhmedMoawd

In [8]:
!unzip /content/GP/Final_dataset.zip

Archive:  /content/GP/Final_dataset.zip
  inflating: Final_dataset/Abdo (1).jpg  
  inflating: Final_dataset/Abdo (2).jpg  
  inflating: Final_dataset/Abdo (3).jpg  
  inflating: Final_dataset/Abdo (4).jpg  
  inflating: Final_dataset/Abdo (5).jpg  
  inflating: Final_dataset/Abdo (6).jpg  
  inflating: Final_dataset/Abdo (7).jpg  
  inflating: Final_dataset/ahmed.jpg  
  inflating: Final_dataset/ahmed1.png  
  inflating: Final_dataset/ahmed2.png  
  inflating: Final_dataset/ahmed3.png  
  inflating: Final_dataset/ahmed4.png  
  inflating: Final_dataset/ahmed5.png  
  inflating: Final_dataset/ahmed6.png  
  inflating: Final_dataset/AhmedMoawd.jpg  
  inflating: Final_dataset/AhmedMoawd1.png  
  inflating: Final_dataset/AhmedMoawd2.png  
  inflating: Final_dataset/AhmedMoawd3.png  
  inflating: Final_dataset/AhmedMoawd4.png  
  inflating: Final_dataset/AhmedMoawd5.png  
  inflating: Final_dataset/AhmedMoawd6.png  
  inflating: Final_dataset/AhmedSaeed.jpeg  
  inflating: Final_dataset/A

In [9]:
class FaceLoading:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (160, 160)
        self.detector = MTCNN()

    def ExtractFace(self, filename):
        img = cv.imread(filename)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        x, y, w, h = self.detector.detect_faces(img)[0]['box']
        x, y = abs(x), abs(y)
        face = img[y:y + h, x:x + w]
        face_arr = cv.resize(face, self.target_size)
        return face_arr

    def LoadFace(self, file_path):
        faces = []
        try:
            single_face = self.ExtractFace(file_path)
            faces.append(single_face)
        except Exception as e:
            pass
        return faces

    def LoadClasses(self):
        x = []
        y = []
        for file_name in os.listdir(self.directory):
            if file_name.endswith('.jpg') or file_name.endswith('.png') or file_name.endswith('.jpeg'):
                file_path = os.path.join(self.directory, file_name)
                faces = self.LoadFace(file_path)
                labels = [file_name] * len(faces)
                print(f"Loaded successfully: {len(labels)}")
                x.extend(faces)
                y.extend(labels)
        return np.asarray(x), np.asarray(y)


    def encode_and_save_embeddings(self):
        X, y = self.LoadClasses()
        embedder = FaceNet()
        embedded_x = [self.GetEmbedding(embedder, img) for img in X]

        # Save embeddings inside the created folder
        for i, (embedding, label) in enumerate(zip(embedded_x, y)):
            np.save(os.path.join(embedding_save_directory, f'{label}_embeddings.npy'), embedding)
        return np.array(embedded_x), np.array(y)



    def GetEmbedding(self, embedder, face_img):
        face_img = face_img.astype('float32')
        face_img = np.expand_dims(face_img, axis=0)
        yhat = embedder.embeddings(face_img)
        return yhat[0]



    def load_or_encode_and_save_embeddings(self):
        embedding_files = [f for f in os.listdir(embedding_save_directory) if f.endswith('.npy')]

        if not embedding_files:
            print("Embeddings not found. Encoding and saving embeddings...")
            return self.encode_and_save_embeddings()
        else:
            print("Embeddings found. Loading embeddings...")
            embedded_x, dataset_labels = [], []
            for file in embedding_files:
                data = np.load(os.path.join(embedding_save_directory, file))
                embedded_x.append(data)
                dataset_labels.append(file.replace('_embeddings.npy', ''))
            embedded_x = np.asarray(embedded_x)
            dataset_labels = np.asarray(dataset_labels)
            return embedded_x, dataset_labels





In [10]:
def preprocess_label(label):
    label = re.sub(r'\d+', '', label)
    label = re.sub(r'\.\w{3,4}|\.', '', label)
    label = re.sub(r'\[.*?\]|\(.*?\)|\{.*?\}', '', label)
    label = re.sub(r'_[\w\d]+.*$', '', label)
    label = label.strip()
    return label



In [11]:
def find_closest_matches(embedded_input, embedded_dataset, dataset_labels, top_n=7):
    distances = []
    outlabels = []
    for input_embedding in embedded_input:
        distances_for_input = [cosine(input_embedding, dataset_embedding) for dataset_embedding in embedded_dataset]
        min_distance_index = np.argmin(distances_for_input)
        distances.append(distances_for_input[min_distance_index])
        outlabels.append(dataset_labels[min_distance_index])

    sorted_indices = np.argsort(distances)
    sorted_distances = np.array(distances)[sorted_indices]
    sorted_labels = np.array(outlabels)[sorted_indices]

    top_matches = [(sorted_labels[i], sorted_distances[i]) for i in range(top_n)]
    return top_matches




In [12]:
def check_for_match(top_matches):
    labels = [preprocess_label(label) for label, _ in top_matches]
    distances = [distance for _, distance in top_matches]
    frequent_labels = Counter(labels).most_common()

    for label, count in frequent_labels:
        if count >= 4:
            label_distances = [distance for orig_label, distance in top_matches if preprocess_label(orig_label) == label]
            avg_similarity = sum(label_distances) / count
            if avg_similarity < 0.5:
                return True, avg_similarity, label

        elif count == 3:
            label_distances = [distance for orig_label, distance in top_matches if preprocess_label(orig_label) == label]
            avg_similarity = sum(label_distances) / count
            if avg_similarity < 0.5:
                return True, avg_similarity, label

    return False, None, None



In [14]:
def main():
    # Initialize FaceLoading object
    faceLoading = FaceLoading('/content/Final_dataset')

    # Load or encode and save embeddings
    embedded_x, dataset_labels = faceLoading.load_or_encode_and_save_embeddings()

    # Load and process new images for comparison
    faceLoading2 = FaceLoading('/content/FinalTest_dataset/AhmedSaeed')
    X1, y1 = faceLoading2.LoadClasses()

    # Compute embeddings for new input images
    embedded_x1 = [faceLoading.GetEmbedding(FaceNet(), img) for img in X1]

    # Find closest matches
    top_matches = find_closest_matches(embedded_x1, embedded_x, dataset_labels, top_n=7)

    # Print top 7 similar images and their similarity scores
    for i, (label, distance) in enumerate(top_matches, 1):
        print(f"Similar Image {i}: Label - {label}, Similarity Score - {distance}")

    # Check for match
    match_found, avg_similarity, matched_label = check_for_match(top_matches)

    if match_found:
        print("Match found with label:", matched_label)
        print("Average cosine similarity:", avg_similarity)
    else:
        print("No match found. Adding embeddings...")
        # Save embeddings of test images as new embeddings
        embedder = FaceNet()
        for i, img in enumerate(X1):
            embedding = faceLoading.GetEmbedding(embedder, img)
            label = y1[i]
            np.save(os.path.join(embedding_save_directory, f'{label}embeddings.npy'), embedding)

if __name__ == "__main__":
    main()

Embeddings found. Loading embeddings...
1/1 [==============================] - 0s 245ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 18ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 163ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 197ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 217ms/step
Loaded successfully: 1
1/1 [==============================] - 1s 639ms/step
Loaded successfully: 1
1/1 [==============================] - 0s 18ms/step
Loaded successfully: 1
1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 2s 2s/step
Similar Image 1: Label - MennaMohamed6.png, Similarity Score - 0.25500160455703735
Similar Image 2: Label - Marawan6.png, Similarity Score - 0.3148021101951599
Similar Image 3: Label - AhmedSaeed.jpeg, Similarity Score - 0.3230445981025696
Similar Image 4: Label - AhmedSaeed0005 (5).png, Similarity Score - 0.37808555364608765
Similar Image 5: Label - Marawan7.jpg, Similarity Score - 0.4569302797317505
Similar Image 6: Label - Abdo (7).jpg, Similarity Score - 0.5902988314628601
Similar Image 7: Label - Khadega (6).png, Similarity Score - 0.7163755893707275
No match found. Adding embeddings...
1/1 [==============================] - 0s 28ms/step
